In [12]:
import random
import os
from datetime import datetime
from tqdm import tqdm
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder

In [13]:
train = pd.read_csv('../../DATA/train.csv')
test  = pd.read_csv('../../DATA/test.csv')

In [14]:

new_column_names = {
    'corporation': 'corp',
    'location': 'loc',
    'supply(kg)': 'supply',
    'price(원/kg)': 'price',
}
train = train.rename(columns=new_column_names)
test = test.rename(columns=new_column_names)
train['timestamp']  = pd.to_datetime(train['timestamp'])
test['timestamp']  = pd.to_datetime(test['timestamp'])

train['newitem'] = train['item'].str.cat([train['corp'], train['loc']], sep=' ')

In [15]:
#가격 시계열 price_data 생성하기 

def before_timeseries_data(train):
    unique_values = train['newitem'].unique()
    # 날짜 범위 생성
    start_date = datetime(2019, 1, 1).date()
    end_date =datetime(2023, 3, 3).date()
    dates = pd.date_range(start_date, end_date, freq='D')

    # 데이터프레임 생성
    price_data = pd.DataFrame(columns=['item'] + dates.strftime('%Y-%m-%d').tolist())
    price_data['item'] = unique_values

    # print(len(price_data)) 39 
    for i in range(len(price_data)) :
        price_data.iloc[i,1:] = train['price'][i*1523 : (i+1)*1523 ]
    price_data['item'] = price_data['item'].astype(str)
    price_data['corp'] = price_data['item'].str[3:4]

    price_data['loca'] = price_data['item'].str[-1:]
    price_data['product'] = price_data['item'].str[:2]
    price_data['product_loca'] =price_data['product'].str.cat(price_data['loca'], sep=' ')
    cols = price_data.columns[-4:]  # 가장 뒤의 3개 열의 열 이름을 선택
    price_data = price_data[cols.tolist() + price_data.columns[:-4].tolist()]  # 열 순서 변경
    
    return price_data

price_data = before_timeseries_data(train)

In [16]:
le = LabelEncoder()
price_data['product_loca'] = le.fit_transform(price_data['product_loca'])
price_df = price_data.copy()

/home/kiwoongyoon/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/kiwoongyoon/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [17]:
def minimum_for_each(data) :
    non_zero_data =[value for value in data if value != 0]
    min_price = np.min(non_zero_data[-18: ])
    return min_price

In [18]:
# interpolation 적용하기 
minforeach = []
for i in range(len(price_df)) :
    # cres , cbas, crds , bcbs , rdcs, bccs 
    # print(price_df.iloc[i, 4])
    if price_df.iloc[i, 4] in ['CR E S', 'CB A S' , 'CR D S', 'BC B S', 'RD C S' , 'BC C S']: continue
    
    prices = list(price_df.iloc[i , 5:])
    minprice= minimum_for_each(np.array(prices)) 
    # if prices[-1] ==0 :        print(price_df.iloc[i, 4] , median_price)  
    

    #각각의 아이템에 대해서 최소의 값을 고른다 
    minforeach.append(( price_df.iloc[i, 4].replace(' ', '_'), minprice))
    

In [19]:
#수정할 제출 데이터 가지고오기 
data = pd.read_csv('../../DATA/SUBMIT/patchTST.csv').rename(columns={'answer': 'price'} )


In [20]:
#최소값 보정 
def fix_min(id, price) :
    item_id = id[:6]
    # return str(item_id)
    for i in range(len(minforeach)):
        if item_id == minforeach[i][0] and price < minforeach[i][1]:
            price= minforeach[i][1]
    if item_id == 'CB_A_S': 
        price = price *0.35
    return price

def change(data) :
    
    data['ID']= data['ID'].astype(str)
    min_fixed = data.apply(lambda row: fix_min(row['ID'],row['price']), axis=1 )
    data['price'] = min_fixed
    data.loc[data['price'] <= 0, 'price'] = 0
    data.loc[data.index % 7 == 1, 'price'] = 0
    data.rename(columns={'price': 'answer'})
    return data 



In [21]:
tosubmit = change(data)

In [22]:
tosubmit.to_csv('../../DATA/SUBMIT/patchTST_minfixed.csv', index=False)